In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'C:/Users/HP/Desktop/Task/Task6/faces/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [ ]:
# pip install opencv-contrib-python 
# for LBPH 

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join


data_path = 'C:/Users/HP/Desktop/Task/Task6/faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)

model  = cv2.face_LBPHFaceRecognizer.create()
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

In [ ]:
import cv2
import numpy as np
import os


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)
a1 = False
a2 = False
while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = model.predict(face)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey Manas", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face', image )
            cv2.imwrite('face.jpg' , image)
            a1 = True      
        else:
            cv2.putText(image, "Cannot Recognize You", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
            cv2.imshow('Face', image )
            a2=True

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Looking for the face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
# pip install pywhatkit

In [ ]:
def whatsapp():
    import pywhatkit
    from datetime import datetime
    currtime = datetime.now()
    hours = int(currtime.strftime("%H"))
    minutes = int(currtime.strftime("%M"))+1
    print("Time at will message will be send = {}:{}".format(hours,minutes))
    pywhatkit.sendwhatmsg('************', 'FACE DETECTED',hours,minutes)

In [ ]:
def mail():
    import getpass
    import smtplib
    fromaddr = 'XXXXXXXXXXXXXXXXXXXX'
    toaddrs  = 'XXXXXXXXXXXXXXXXXXXXX' 
    msg = 'Someone has been detected'  
    username = 'XXXXXXXXXXXXXXXXXXXX'  
    password = getpass.getpass()
    server = smtplib.SMTP('smtp.gmail.com', 587)  
    server.ehlo()
    server.starttls()
    server.login(username, password)  
    server.sendmail(fromaddr, toaddrs, msg)  
    server.quit()

In [ ]:
import subprocess
import time
import os
def instance_launch():
    i_id = subprocess.getoutput("aws ec2 run-instances  --region ap-south-1  --image-id ami-0ad704c126371a549  --instance-type t2.micro --count 1  --key-name awskey  --query Instances[*].[InstanceId] --output text")
    print("EC2 Instance with instance id : {} created".format(i_id))

def volume_creation():
    v_id = subprocess.getoutput("aws ec2 create-volume  --region ap-south-1  --volume-type gp2 --size 5 --availability-zone ap-south-1a --query VolumeId --output text")
    print("Created 5GB EBS Volume with volume id : {} created".format(v_id))
    time.sleep(15)            
                
def attach():
    i_id = input("Enter instance id: ")
    v_id = input("Enter volume id: ")
    os.system("aws ec2 attach-volume --device /dev/xvdf --instance-id  " + i_id +" --volume-id " + v_id + " ")
    print("EBS volume Attached successfully to the newly created instance")


In [ ]:
if a1:
    whatsapp()
    mail()
if a2:
    instance_launch()
    volume_creation()
    attach()